## Transshipment Problem

In [1]:
from pyomo.environ import *

# Create model
model = ConcreteModel()

# Define sets
Nodes = [1, 2, 3, 4, 5, 6, 7, 8]

# Define arcs and their costs
Arcs = [
    (1, 3), (1, 4),
    (2, 3), (2, 4),
    (3, 5), (3, 6), (3, 7), (3, 8),
    (4, 5), (4, 6), (4, 7), (4, 8),
    (2, 8),
    (7, 8)
]

Costs = {
    (1, 3): 2, (1, 4): 3,
    (2, 3): 3, (2, 4): 1,
    (3, 5): 2, (3, 6): 6, (3, 7): 3, (3, 8): 6,
    (4, 5): 4, (4, 6): 4, (4, 7): 6, (4, 8): 5,
    (2, 8): 4,
    (7, 8): 1
}

# Define variables
model.x = Var(Arcs, domain=NonNegativeReals)

# Objective function
model.obj = Objective(expr=sum(Costs[i, j] * model.x[i, j] for (i, j) in Arcs), sense=minimize)

# Constraints
model.constraints = ConstraintList()

# Origin nodes: 1 and 2
model.constraints.add(model.x[1, 3] + model.x[1, 4] <= 600)
model.constraints.add(model.x[2, 3] + model.x[2, 4] + model.x[2, 8] <= 400)

# Transshipment nodes: 3 and 4 (flow in = flow out)
model.constraints.add(
    -model.x[1, 3] - model.x[2, 3] + model.x[3, 5] + model.x[3, 6] + model.x[3, 7] + model.x[3, 8] == 0
)
model.constraints.add(
    -model.x[1, 4] - model.x[2, 4] + model.x[4, 5] + model.x[4, 6] + model.x[4, 7] + model.x[4, 8] == 0
)

# Destination nodes
model.constraints.add(model.x[3, 5] + model.x[4, 5] == 200)
model.constraints.add(model.x[3, 6] + model.x[4, 6] == 150)
model.constraints.add(model.x[3, 7] + model.x[4, 7] - model.x[7, 8] == 350)
model.constraints.add(model.x[3, 8] + model.x[4, 8] + model.x[2, 8] + model.x[7, 8] == 300)

# Solve
solver = SolverFactory('glpk')  # or 'cbc' or any installed solver
solver.solve(model)

# Print results
print("Optimal solution:")
for (i, j) in Arcs:
        print(f"x[{i},{j}] = {model.x[i,j].value}")

print(f"Total Cost = {model.obj.expr():.2f}")

Optimal solution:
x[1,3] = 550.0
x[1,4] = 50.0
x[2,3] = 0.0
x[2,4] = 100.0
x[3,5] = 200.0
x[3,6] = 0.0
x[3,7] = 350.0
x[3,8] = 0.0
x[4,5] = 0.0
x[4,6] = 150.0
x[4,7] = 0.0
x[4,8] = 0.0
x[2,8] = 300.0
x[7,8] = 0.0
Total Cost = 4600.00
